In [2]:
import os
import shutil
from creds import OPENAI_KEY
from application_generator import JobApplicationBuilder
from utils import LatexToolBox, text_to_pdf
from models import ChatGPT
from prompts.resume_section_prompts import RESUME_WRITER_PERSONA
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

llm = ChatGPT(
    api_key=OPENAI_KEY,
    model="gpt-4o",
    system_prompt=RESUME_WRITER_PERSONA,
    max_output_tokens=None,
    temperature=0.7
)

generator = JobApplicationBuilder(
    llm=llm,
    resume_template_file="slightly_less_basic_template.tex"
    )

job_content_path = r"input_data/Adobe Intern Data Science in San Jose R149568.pdf"
resume_pdf_path = r"input_data/full_resume_contents_20250120.pdf"
#resume_pdf_path = r"input_data/Grad School Resume.pdf"
job_details_dict, job_details_filepath = generator.extract_job_content(job_content_path)
resume_dict = generator.user_data_extraction(user_data_path=resume_pdf_path)
resume_details_dict, resume_details_filepath = generator.generate_resume_json(job_content=job_details_dict,
                                                                              user_data=resume_dict)

resume_latex, resume_tex_path = generator.resume_json_to_resume_tex(resume_details=resume_details_dict)
print("Done generating resume tex file")


custom_letter_instructions = input("Enter any custom application instructions for the cover letter: ")
cover_letter, cover_letter_txt_path = generator.generate_cover_letter(job_details=job_details_dict,
                                                                      user_data=resume_details_dict,
                                                                      custom_instructions=custom_letter_instructions,
                                                                      need_pdf=False)

# check that the fonts in the resume tex are available and flag the ones not installed
resume_tex_fonts, _ = LatexToolBox.extract_tex_font_dependencies(resume_tex_path)
font_statuses = LatexToolBox.check_fonts_installed(resume_tex_fonts)
if not all([v for k, v in font_statuses.items()]):
    [print(f"{k} not installed") for k, v in font_statuses.items() if not v]

Job Details JSON generated at: output_files/Adobe/DataScienceSumm_20250120205150/JD.json

Fetching user data...

Generating Resume Details...
Processing Resume's WORK_EXPERIENCE Section...
Processing Resume's PROJECTS Section...
Processing Resume's SKILL_SECTION Section...
Processing Resume's EDUCATION Section...
Processing Resume's CERTIFICATIONS Section...
Processing Resume's ACHIEVEMENTS Section...
Done generating resume tex file
Cover Letter generated at:  output_files/Adobe/DataScienceSumm_20250120205150/cover_letter.txt


Matplotlib is building the font cache; this may take a moment.


Edit the resume latex and cover letter text before running the next cell.

In [2]:
LatexToolBox.compile_latex_to_pdf(tex_file_path=resume_tex_path)
print(f"Resume PDF is saved at {resume_tex_path.replace('.tex', '.pdf')}")

cover_letter_pdf_path = cover_letter_txt_path.replace('.txt', '.pdf')
cover_letter_pdf_path = text_to_pdf(cover_letter, cover_letter_pdf_path)
print(f"Cover Letter PDF is saved at {cover_letter_pdf_path}")

PDF successfully saved to /Users/aaronrdankert/projects/job_hunter_toolbox/output_files/AsmlUs/InternHmiCustom_20250117203226
Resume PDF is saved at output_files/AsmlUs/InternHmiCustom_20250117203226/resume.pdf
Cover Letter PDF is saved at output_files/AsmlUs/InternHmiCustom_20250117203226/cover_letter.pdf


In [3]:
# move the job content file to the output folder
shutil.move(job_content_path, generator.get_job_doc_path())

# move the job_content_path to the output folder
shutil.move(resume_pdf_path, generator.get_job_doc_path())

'output_files/AsmlUs/InternHmiCustom_20250117203226/full_resume_contents_20250110.pdf'